In [ ]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt 
import geopandas as gpd
import shapely.geometry
from shapely.geometry import Point,MultiPolygon,Polygon
from scipy.stats import zscore

import math

In [ ]:
# loading up shapefiles

# county visualization shape file
gdf = gpd.read_file("cb_2018_us_county_500k/cb_2018_us_county_500k.shp")
#print(gdf.crs)
gdf.to_crs('EPSG:4326') # converts to WSG84
gdf['county_id'] = pd.to_numeric(gdf.apply(lambda x: str(x['STATEFP'])+str(x['COUNTYFP']),axis = 1))


state_gdf = gpd.read_file("cb_2018_us_state_500k/cb_2018_us_state_500k.shp")
state_gdf.to_crs('EPSG:4326') # converts to WSG84

#index_df = pd.read_excel('index.xlsx')
#index_df = index_df.rename({'Unnamed: 0.1':'county_id'},axis = 1)
#gdf = gdf.merge(index_df[['county_id','security_index']], how = 'left', on = 'county_id')



# hunting land data
huntingland = gpd.read_file("FWS_National_2022_-_2023_Hunt_Units/FWS_NWRS_HQ_PubHuntUnits.shp")
#print(huntingland['geometry'].crs) # this checks the gemoetry type
huntingland = huntingland.to_crs('EPSG:4326') # converts to WSG84
huntingland = huntingland[huntingland['Huntable']=='Yes']

# Hydro data
water_gdf = gpd.read_file('Watershapefile/hydropol.shp')


# Food point data

csa_names = ['listing_id','location_x','location_y',
             'acceptedpayment','FNAP','products','product_fruit',
             'product_vegetables','product_fishseafood',
             'product_poultryfowl_otherdesc','product_poultryfowl','product_redmeat']

farmersmarket_names = ['listing_id','location_x','location_y','acceptedpayment','FNAP']

foodhubs_names = ['listing_id','location_x','location_y','acceptedpayment','FNAP','products']



farm_mark = pd.read_excel('PointData/FarmersMarket.xlsx')
farm_mark = farm_mark[farmersmarket_names]
farm_mark_gdf = gpd.GeoDataFrame(farm_mark, 
                                 geometry=gpd.points_from_xy(farm_mark.location_x, farm_mark.location_y),
                                 crs="EPSG:4269") # not sure but they are long and lat
farm_mark_gdf = farm_mark_gdf[farm_mark_gdf['geometry'].is_empty != True]

csa = pd.read_excel('PointData/CSA.xlsx')
csa = csa[csa_names]
csa_gdf = gpd.GeoDataFrame(csa,
                           geometry=gpd.points_from_xy(csa.location_x, csa.location_y),
                           crs="EPSG:4269")
csa_gdf = csa_gdf[csa_gdf['geometry'].is_empty != True]


foodhub = pd.read_excel('PointData/Foodhub.xlsx')
foodhub = foodhub[foodhubs_names]
foodhub_gdf = gpd.GeoDataFrame(foodhub,
                               geometry=gpd.points_from_xy(foodhub.location_x, foodhub.location_y),
                               crs="EPSG:4269")
foodhub_gdf = foodhub_gdf[foodhub_gdf['geometry'].is_empty != True]


del csa,foodhub,farm_mark

farm_mark_gdf = farm_mark_gdf.to_crs("EPSG:4326") # to degrees
foodhub_gdf = foodhub_gdf.to_crs("EPSG:4326")
csa_gdf = csa_gdf.to_crs("EPSG:4326")

#Vegetables
fruit_veglist = ['product_vegetables','product_fruit']
for plant in fruit_veglist:
    csa_gdf[plant] = csa_gdf[plant].isnull()
    csa_gdf[plant] = csa_gdf.apply(lambda x: 1 if x[plant] == False else 0, axis = 1)

#Meatlist
meatlist = ['product_fishseafood','product_poultryfowl_otherdesc','product_poultryfowl','product_redmeat']
for meat in meatlist:
    csa_gdf[meat] = csa_gdf[meat].isnull()
    csa_gdf[meat] = csa_gdf.apply(lambda x: 1 if x[meat] == False else 0, axis = 1)

csa_gdf['plant'] = csa_gdf[fruit_veglist].sum(axis=1)
csa_gdf['meat'] = csa_gdf[meatlist].sum(axis=1)

# this is for meat and plant 

cols = ['listing_id','geometry']

meat_gdf = csa_gdf[csa_gdf['meat']>=1]
plant_gdf = csa_gdf[csa_gdf['plant']>=1]

meat_gdf = pd.concat([meat_gdf[cols],farm_mark_gdf[cols]],axis = 0)
plant_gdf = pd.concat([plant_gdf[cols],farm_mark_gdf[cols]],axis = 0)


syn_family_df = pd.read_excel('synthetic_family.xlsx')
syn_family_df['geometry_string'] = syn_family_df['geometry'].apply(lambda x: x.strip('POINT').strip(' (').strip(')'))
syn_family_df['lon'] =  syn_family_df['geometry_string'].apply(lambda x: x.split(' ')[0])
syn_family_df['lat'] =  syn_family_df['geometry_string'].apply(lambda x: x.split(' ')[1])
syn_family_df = syn_family_df[['county_id','lon','lat']]
syn_family_gdf = gpd.GeoDataFrame(syn_family_df, 
                                  geometry = gpd.points_from_xy(syn_family_df.lon,syn_family_df.lat),
                                  crs = 'EPSG:4326')           
del syn_family_df

In [ ]:
angelos = pd.read_csv('New_Capacity_Variables_Food_Access_Angelos.csv')
angelos.head()

In [ ]:
water = pd.read_csv('results_waterdata.csv')
water = water.rename({'distance':'distance_water'},axis = 1)
#water.head()

In [ ]:
raw_data = pd.read_excel('master_file.xlsx')
college = pd.read_csv('College_pctg.csv')
raw_data = raw_data.merge(college[['fips','value']], left_on = 'county_id', right_on = 'fips', how = 'left')
raw_data = raw_data.merge(water, how = 'left', on = 'county_id')

raw_data = raw_data.rename({'value':'college_pct'},axis = 1)
#raw_data.head()

In [ ]:
raw_data = raw_data.merge(angelos[['supercpth','convspth','grocpth','ffrpth','fips']],
                       left_on = 'county_id',right_on = 'fips', how = 'left')

In [ ]:
index_data = raw_data

#index_data = pd.read_csv('index_data.csv')
# county_pop = pd.read_csv('county_pop.csv')
# index_data = index_data.merge(county_pop[['county_id','ESTIMATESBASE2020']], how = 'left', on = 'county_id')
titlelist = ['distance_meat','distance_plant',
            'distance_foodhub','distance_hunting','distance_water',
            'food_insecurity_rate','pct_laccess_hhnv','college_pct',
             'supercpth','convspth','grocpth','ffrpth']

# index_data['ag_acres'] = index_data['ag_acres']/index_data['ESTIMATESBASE2020']
index_data[titlelist] = index_data[titlelist]/index_data[titlelist].max()
#index_data.head()
index_data=index_data.fillna(99999)


In [ ]:
# maximum value of 5
# big numbers = insecurity
index_data['quality_score'] = index_data[['distance_meat','distance_plant',
                                          'distance_foodhub','distance_hunting','distance_water']].sum(axis = 1)
# maximum value of 4
# big numbers = security
index_data['quantity_score'] = index_data[['supercpth','convspth','grocpth','ffrpth']].sum(axis = 1)

# maximum value 3
# big numbers mean insecurity
index_data['college_pct'] = index_data['college_pct']*(-1)
index_data['demographic_score'] = index_data[['food_insecurity_rate','pct_laccess_hhnv','college_pct']].sum(axis = 1)

index_data['quality_score'] = (index_data[['quality_score']]/5)*(2.5/8) # value of 1
index_data['quantity_score'] = -1*(index_data[['quantity_score']]/4)*(2.5/8) # value of 1
index_data['demographic_score'] = ((index_data[['demographic_score']]+1)/3)*(3/8) # value of -1

# index is food security small values are bad low security high values are good lots of security
# so thats what I'll do
index_data['index'] = (index_data[['quality_score','quantity_score','demographic_score']].sum(axis = 1, min_count=1)+(2.5/8))*(100)


In [ ]:
gdf_color_plot['index'].describe()

In [ ]:
gdf_color_plot = gdf.merge(index_data[['county_id','index']], how = 'left', right_on = 'county_id',left_on = 'county_id')
gdf_color_plot['index_b'] = gdf_color_plot['index'].apply(lambda x: 1 if (x>100 or x < 0) else 0)
gdf_color_plot['index_b'] = gdf_color_plot.apply(lambda x: 1 if x['county_id'] == 48301 else x['index_b'], axis = 1)
gdf_color_plot['index'] = gdf_color_plot['index'].apply(lambda x: np.nan if (x>100 or x < 0) else x)
gdf_gray = gdf_color_plot[gdf_color_plot['index_b']==1]


In [ ]:
# klls loving county texas
gdf_color_plot=gdf_color_plot[gdf_color_plot['county_id']!= 48301]

In [ ]:
gdf_gray.head()

In [ ]:
# #synth_family

fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf.plot(ax = ax,color = 'white' , edgecolor = 'black',
         zorder = 1, linewidth=.05,
         legend = True ) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.1) # this is the US map at state level


syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
# huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
# meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
# foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
# plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
# water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')


ax.axis('off')

ax.set_xlim(-128, -60)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(22,50)
plt.savefig('USA_SYN_FAM_FINAL.jpg',dpi = 1200)

In [ ]:
# #synth_family

fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf.plot(ax = ax,color = 'white' , edgecolor = 'black',
         zorder = 1, linewidth=.05,
         legend = True ) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.1) # this is the US map at state level


syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
# huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
# meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
# foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
# plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
# water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')


ax.axis('off')

ax.set_xlim(-187,-125)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(48,74)
plt.savefig('USA_MAP_HH_AK.jpg',dpi = 4800)

In [ ]:
# #synth_family

fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf.plot(ax = ax,color = 'white' , edgecolor = 'black',
         zorder = 1, linewidth=.05,
         legend = True ) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.1) # this is the US map at state level


syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
# huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
# meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
# foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
# plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
# water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')


ax.axis('off')

ax.set_xlim(-163,-153)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(18,23)
plt.savefig('USA_MAP_HI_HH.jpg',dpi = 4800)

In [ ]:
# Points

In [ ]:
# #synth_family

fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf.plot(ax = ax,color = 'white' , edgecolor = 'black',
         zorder = 1, linewidth=.05,
         legend = True ) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.1) # this is the US map at state level


syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')


ax.axis('off')

ax.set_xlim(-128, -60)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(22,50)
plt.savefig('USA_MAP_point.jpg',dpi = 1200)

In [ ]:
# #synth_family

fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf.plot(ax = ax,color = 'white' , edgecolor = 'black',
         zorder = 1, linewidth=.05,
         legend = True ) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.1) # this is the US map at state level


syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')


ax.axis('off')

ax.set_xlim(-187,-125)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(48,74)
plt.savefig('USA_MAP_point_AK.jpg',dpi = 4800)

In [ ]:
# #synth_family

fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf.plot(ax = ax,color = 'white' , edgecolor = 'black',
         zorder = 1, linewidth=.05,
         legend = True ) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.1) # this is the US map at state level


syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')


ax.axis('off')

ax.set_xlim(-163,-153)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(18,23)
plt.savefig('USA_MAP_point_HI.jpg',dpi = 4800)

In [ ]:
# INDEX

In [ ]:
# #synth_family
vmin = 40 #gdf_color_plot['index'].min()
vmax = 55 #gdf_color_plot['index'].max()


fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)
# use winter_r for counterfactual

# use 
gdf_color_plot.plot(column = 'index' ,ax = ax, cmap = 'hot_r' , edgecolor = 'black',
                    zorder = 1, linewidth=.05,
                    legend = True ,legend_kwds={'shrink': 0.55,"fmt": "{:.0f}"},
                   vmin = vmin, vmax = vmax) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.2) # this is the US map at state level

# syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
# huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
# meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
# foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
# plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
# water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')
gdf_gray.plot(ax = ax, color = 'grey' , edgecolor = 'black',
         zorder = 2, linewidth=.05) 

#ax.tick_params(labelsize=15)                         
ax.axis('off')

ax.set_xlim(-128, -60)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(22,50)
#plt.savefig('counter_Factual.jpg',dpi = 1200)

In [ ]:
# #synth_family
vmin = 40 #gdf_color_plot['index'].min()
vmax = 55 #gdf_color_plot['index'].max()


fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf_color_plot.plot(column = 'index' ,ax = ax, cmap = 'hot_r' , edgecolor = 'black',
                    zorder = 1, linewidth=.05,
                    legend = True ,legend_kwds={'shrink': 0.55,"fmt": "{:.0f}"},
                   vmin = vmin, vmax = vmax) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.2) # this is the US map at state level

# syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
# huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
# meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
# foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
# plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
# water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')
gdf_gray.plot(ax = ax, color = 'grey' , edgecolor = 'black',
         zorder = 2, linewidth=.05) 

#ax.tick_params(labelsize=15)   
ax.axis('off')

ax.set_xlim(-187,-125)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(48,74)
plt.savefig('USA_MAP_AK_index.jpg',dpi = 1200)

In [ ]:
# #synth_family
vmin = 40 #gdf_color_plot['index'].min()
vmax = 55 #gdf_color_plot['index'].max()


fig, ax = plt.subplots()
# ax.set_title('Point Visualization',fontsize=12)

gdf_color_plot.plot(column = 'index' ,ax = ax, cmap = 'hot_r' , edgecolor = 'black',
                    zorder = 1, linewidth=.05,
                    legend = True ,legend_kwds={'shrink': 0.55,"fmt": "{:.0f}"},
                   vmin = vmin, vmax = vmax) # this is the US map at county level

state_gdf.plot(ax = ax, color = 'none' , edgecolor = 'black',
         zorder = 2, linewidth=.2) # this is the US map at state level

# syn_family_gdf.plot(zorder = 3, ax = ax, markersize = .01, color = 'grey',marker = 'x')
# huntingland.plot(zorder = 4, ax = ax, markersize = .01, color = 'orange')
# meat_gdf.plot(zorder = 5, ax = ax, markersize = .01, color = 'red',marker = 'x')
# foodhub_gdf.plot(zorder = 6, ax = ax, markersize = .01, color = 'brown',marker = 'x')
# plant_gdf.plot(zorder = 7, ax = ax, markersize = .01, color = 'green',marker = 'x')
# water_gdf.plot(zorder = 8, ax = ax, markersize = .01, color = 'blue')
gdf_gray.plot(ax = ax, color = 'grey' , edgecolor = 'black',
         zorder = 2, linewidth=.05) 

#ax.tick_params(labelsize=15)   
ax.axis('off')

ax.set_xlim(-163,-153)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(18,23)
plt.savefig('USA_MAP_HI_index.jpg',dpi = 1200)